<a href="https://colab.research.google.com/github/yaswhar/learning-genesis-sim/blob/main/Control%20Your%20Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install genesis-world -q
!pip install mediapy -q

In [ ]:
import os
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
ICD_CONFIG_CONTENT = """{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
"""
with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write(ICD_CONFIG_CONTENT)

In [ ]:
import genesis as gs
import numpy as np
import mediapy as media
from tqdm.notebook import tqdm

gs.init(backend=gs.cuda)